 <img src="https://ibm.box.com/shared/static/9gegpsmnsoo25ikkbl4qzlvlyjbgxs5x.png" width="400" />

## Coursera Capstone - Week 3

This notebook will be completed as a part of the Capstone Project that is in the final module of Applied Data Science Capstone Course offered by IBM. The specilizaiion takes us from an in-depth introduction and an intuitive understandung of data analysis to detailed exploration of ways to extract information from datasets (both small and large-scale) and manipulating and visualizing them.

### Submission Week 3: Segmenting and Clustering Neighborhoods in Toronto:

**Note**: The work for all three parts will be completed in this same notebook starting below.
Each part will have a title heading and the solution following it.

### Part 1. Scraping the webpage to Pandas dataframe and preprocessing:

In [1]:
# importing libraries...

import folium
import html5lib
import json
import matplotlib.cm as cm
import matplotlib.colors as colors
import numpy as np

import pandas as pd
from pandas.io.json import json_normalize
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import geocoder
import pgeocode
from geopy.geocoders import Nominatim
import requests
from sklearn.cluster import KMeans

In [6]:
# scraping using pandas
toronto_df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M',
                  flavor='html5lib', )[0]
toronto_df.rename(columns={'Postcode':'PostalCode'}, inplace=True)
toronto_df.head()

,PostalCode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


### 2. Data wrangling:

Here we'll be performing the required preprocessing tasks for the dataframe.

**Ignoring entries with Borough that are not assigned:**

First, we replace the Boroughs that are not assigned with NaNs,
then drop those rows..

In [7]:
toronto_df['Borough'].replace('Not assigned', np.nan, inplace=True)
toronto_df = toronto_df.dropna().reset_index(drop=True)

toronto_df.head()

,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor


**Combining Neighbourhoods with same Postcode:**

Here we use Groupby and apply list method to join together elements in Neighbourhoods

Because of applying list method after Groupby below, each element in Neighborhood
column will be a list. So we unlist all elements in Neighbourhood column after that to have a clean dataframe..

In [8]:
toronto_df = toronto_df.groupby(['PostalCode', 'Borough'])['Neighbourhood'].apply(list).reset_index(name='Neighbourhood')
toronto_df['Neighbourhood'] = toronto_df['Neighbourhood'].apply(lambda x: ', '.join(x))

toronto_df.head()

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


**Dealing with Neuighbourhoods that are not assigned:**

Here we copy the Borough name to Neighbourhood if Neighbourhood
is not assigned

In [9]:
# This is an entry with assigned Borough but not assigned Neighbourhood
toronto_df[toronto_df['Neighbourhood']=='Not assigned']

,PostalCode,Borough,Neighbourhood
93,M9A,Queen's Park,Not assigned


In [10]:
for index, neighbourhoods in enumerate(toronto_df['Neighbourhood']):
    if neighbourhoods=='Not assigned':
        toronto_df.loc[index]['Neighbourhood'] = toronto_df.loc[index]['Borough']

toronto_df[toronto_df['Neighbourhood']=='Not assigned']  # should be empty

,PostalCode,Borough,Neighbourhood


**Shape of the dataframe:**

And finally to print number of rows and columns of dataframe....

In [11]:
toronto_df.shape

(103, 3)

### Part 2: Get Latitude and Longitude of each Neighbourhood:

Here, to get the Latitude and Longitude coordinates of each Neighbourhood, a library named **pgeocode** is used, which offers a very simple API. For countries that are supported by this library, it only requires the Postal code and returns their respective latitudes and longitudes.

In [12]:
latitudes = []
longitudes = []
postcodes = []
for i in toronto_df['PostalCode']:
    results = pgeocode.Nominatim('ca').query_postal_code(i)
    latitudes.append(results['latitude'])
    longitudes.append(results['longitude'])
    postcodes.append(i)  # postcodes to be used for joining reference

In [13]:
# Creating temporary dataframe containing location coordinates with postal codes
temp_df = pd.DataFrame(list(zip(postcodes, latitudes, longitudes)), columns=['PostalCode', 'Latitude', 'Longitude'])
temp_df.head()

,PostalCode,Latitude,Longitude
0,M1B,43.8113,-79.1930
1,M1C,43.7878,-79.1564
2,M1E,43.7678,-79.1866
3,M1G,43.7712,-79.2144
4,M1H,43.7686,-79.2389


In [14]:
# Now to merge the dataframes based on postal codes
toronto_df = pd.merge(toronto_df, temp_df, on='PostalCode', how='inner', suffixes=('',''))
toronto_df.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.8113,-79.1930
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.7878,-79.1564
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.7678,-79.1866
3,M1G,Scarborough,Woburn,43.7712,-79.2144
4,M1H,Scarborough,Cedarbrae,43.7686,-79.2389


Sanity check to see if pgeocode got all the latitudes and longitudes values and not NaNs:

In [15]:
np.argwhere(np.isnan(latitudes))

array([[86]])

Bummer!! We have one NaN value at index 86. We will use the given csv file to fill that..

In [31]:
latitude, longitude = pd.read_csv('https://cocl.us/Geospatial_data').loc[86][1:]
print(latitude, longitude)

43.6369656 -79.61581899999999


In [32]:
toronto_df.loc[86, 'Latitude'] = latitude
toronto_df.loc[86, 'Longitude'] = longitude

#### And finally, we check for Null values once again:

In [33]:
toronto_df.isnull().sum()

PostalCode       0
Borough          0
Neighbourhood    0
Latitude         0
Longitude        0
dtype: int64

Perfect..!

On to the next task..

### Part 3: Exploring and Clustering Neighbourhoods:

Since we grouped Neighbourhoods under the same postal code together, we have latitudes and longitudes for each Postal code, not the Neighbourhood. Let's plot those postal areas using Folium first..

In [35]:
address = 'Toronto, Ontario, Canada'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(f'The geographical coordinates of Toronto are: {latitude}, {longitude}')

The geographical coordinates of Toronto are: 43.653963, -79.387207


In [18]:
# create map of Toronto postal codes using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_df['Latitude'], toronto_df['Longitude'], toronto_df['Borough'], toronto_df['Neighbourhood']):
    label = '{}: {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)

map_toronto

Now, we use FourSquare API to segment the postal codes based on venues..

Initializing parameters needed for Foursquare:

In [36]:
CLIENT_ID = ''
CLIENT_SECRET = ''
VERSION = ''
radius = 500
LIMIT = 100

To cluster together, we need venues and then features from them. We will use the function created for Manhattan Neighbourhoods to get nearby venues:

In [37]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Postal Code', 
                  'Postal Code Latitude', 
                  'Postal Code Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [39]:
toronto_venues = getNearbyVenues(names=toronto_df['PostalCode'],
                                 latitudes=toronto_df['Latitude'],
                                 longitudes=toronto_df['Longitude'])

M1B
M1C
M1E
M1G
M1H
M1J
M1K
M1L
M1M
M1N
M1P
M1R
M1S
M1T
M1V
M1W
M1X
M2H
M2J
M2K
M2L
M2M
M2N
M2P
M2R
M3A
M3B
M3C
M3H
M3J
M3K
M3L
M3M
M3N
M4A
M4B
M4C
M4E
M4G
M4H
M4J
M4K
M4L
M4M
M4N
M4P
M4R
M4S
M4T
M4V
M4W
M4X
M4Y
M5A
M5B
M5C
M5E
M5G
M5H
M5J
M5K
M5L
M5M
M5N
M5P
M5R
M5S
M5T
M5V
M5W
M5X
M6A
M6B
M6C
M6E
M6G
M6H
M6J
M6K
M6L
M6M
M6N
M6P
M6R
M6S
M7A
M7R
M7Y
M8V
M8W
M8X
M8Y
M8Z
M9A
M9B
M9C
M9L
M9M
M9N
M9P
M9R
M9V
M9W


In [40]:
len(toronto_venues['Postal Code'].unique())

101

So, out of 103 Postal areas, we were able to get venues for 101 of them..

In [41]:
# Size of the resulting dataframe..
print(toronto_venues.shape)
toronto_venues.head()

(2266, 7)


,Postal Code,Postal Code Latitude,Postal Code Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M1B,43.8113,-79.1930,Canadian Appliance Source Whitby,43.808353,-79.191331,Home Service
1,M1E,43.7678,-79.1866,Chick-N-Joy,43.768752,-79.187982,Fried Chicken Joint
2,M1E,43.7678,-79.1866,Bulk Barn,43.771342,-79.184341,Food & Drink Shop
3,M1E,43.7678,-79.1866,Little Caesars Pizza,43.769046,-79.184386,Pizza Place
4,M1E,43.7678,-79.1866,LCBO,43.771462,-79.184384,Liquor Store


Let's see how many venues were returned for each Postal code:

In [42]:
toronto_venues.groupby('Postal Code', as_index=False).count()[['Postal Code', 'Venue']]

,Postal Code,Venue
0,M1B,1
1,M1E,31
2,M1G,1
3,M1H,3
4,M1J,2
5,M1K,17
6,M1L,10
7,M1M,9
8,M1N,4
9,M1P,4


And to see the unique categories:

In [43]:
print('There are {} unique categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 257 unique categories.


### Analyzing each Postal Code:

In [44]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Postal Code'] = toronto_venues['Postal Code'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Postal Code,Accessories Store,Afghan Restaurant,Airport,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Basketball Stadium,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bistro,Board Shop,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,College Stadium,College Theater,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hostel,Hotel,Hotel Bar,Housing Development,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Leather Goods Store,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Photography Studio,Pilates Studio,Pizza Place,Platform,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Post Office,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skating Rink,Smoke Shop,Snack Place,Soccer Field,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,M1B,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,

Examining the size of the dataframe:

In [45]:
toronto_onehot.shape

(2266, 258)

Next, we group the rows together and take the mean of the frequency of occurenece to get our feature matrix that will be used further:

In [47]:
toronto_grouped = toronto_onehot.groupby('Postal Code').mean().reset_index()
toronto_grouped.head()

,Postal Code,Accessories Store,Afghan Restaurant,Airport,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Basketball Stadium,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bistro,Board Shop,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,College Stadium,College Theater,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hostel,Hotel,Hotel Bar,Housing Development,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Leather Goods Store,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Photography Studio,Pilates Studio,Pizza Place,Platform,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Post Office,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skating Rink,Smoke Shop,Snack Place,Soccer Field,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,M1B,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0

To see the new size:

In [48]:
toronto_grouped.shape

(101, 258)

Now, to print each Postal code along with top most common venues:

In [49]:
num_top_venues = 5

for code in toronto_grouped['Postal Code']:
    print("----"+code+"----")
    temp = toronto_grouped[toronto_grouped['Postal Code'] == code].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----M1B----
                       venue  freq
0               Home Service   1.0
1          Accessories Store   0.0
2              Metro Station   0.0
3         Mexican Restaurant   0.0
4  Middle Eastern Restaurant   0.0


----M1E----
                  venue  freq
0           Pizza Place  0.16
1         Grocery Store  0.06
2              Pharmacy  0.06
3  Fast Food Restaurant  0.06
4           Coffee Shop  0.06


----M1G----
                       venue  freq
0          Korean Restaurant   1.0
1          Accessories Store   0.0
2                Men's Store   0.0
3         Mexican Restaurant   0.0
4  Middle Eastern Restaurant   0.0


----M1H----
                        venue  freq
0  Construction & Landscaping  0.33
1                       Trail  0.33
2                      Lounge  0.33
3         Monument / Landmark  0.00
4                 Music Venue  0.00


----M1J----
                             venue  freq
0                              Spa   0.5
1                    Grocery Store

#### Putting that into a *pandas* dataframe

First, let's write a function to sort the venues in descending order.

In [50]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Let's create the new dataframe and display top 10 venues..

In [64]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Postal Code']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
toronto_venues_sorted = pd.DataFrame(columns=columns)
toronto_venues_sorted['Postal Code'] = toronto_grouped['Postal Code']

for ind in np.arange(toronto_grouped.shape[0]):
    toronto_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

toronto_venues_sorted.head()

,Postal Code,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Home Service,Yoga Studio,Doner Restaurant,Flea Market,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Falafel Restaurant
1,M1E,Pizza Place,Coffee Shop,Pharmacy,Grocery Store,Fast Food Restaurant,Thrift / Vintage Store,Juice Bar,Fried Chicken Joint,Beer Store,Electronics Store
2,M1G,Korean Restaurant,Yoga Studio,Food,Flea Market,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Falafel Restaurant
3,M1H,Construction & Landscaping,Lounge,Trail,Event Space,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Falafel Restaurant
4,M1J,Spa,Grocery Store,Event Space,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Falafel Restaurant,Dog Run


### Clustering:

On to clustering Postal areas now.. We will run K-means to cluster the areas into 6 clusters.

In [65]:
# set number of clusters
kclusters = 6

toronto_grouped_clustering = toronto_grouped.drop('Postal Code', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 1, 5, 1, 1, 1, 1, 1, 1, 1], dtype=int32)

Creating a dataframe that includes cluster labels as well as top venues.

First, let's rename the column we'll be using to join the two dataframes..

In [66]:
toronto_venues_sorted.rename(columns={'Postal Code': 'PostalCode'}, inplace=True)

**Note:** M1C and M1X Postal areas had no venues hence no Cluster assignment. So, when joining the two dataframes (one with Venues data and other with Borough and Neighbourhood data), we will use **inner join** so to only include entries that had both venue data+cluster labels and borough+neighbourhood data.

In [67]:
toronto_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_df

toronto_merged = toronto_merged.join(toronto_venues_sorted.set_index('PostalCode'), on='PostalCode', how='inner').reset_index(drop=True)

toronto_merged.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge, Malvern",43.8113,-79.1930,2,Home Service,Yoga Studio,Doner Restaurant,Flea Market,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Falafel Restaurant
1,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.7678,-79.1866,1,Pizza Place,Coffee Shop,Pharmacy,Grocery Store,Fast Food Restaurant,Thrift / Vintage Store,Juice Bar,Fried Chicken Joint,Beer Store,Electronics Store
2,M1G,Scarborough,Woburn,43.7712,-79.2144,5,Korean Restaurant,Yoga Studio,Food,Flea Market,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Falafel Restaurant
3,M1H,Scarborough,Cedarbrae,43.7686,-79.2389,1,Construction & Landscaping,Lounge,Trail,Event Space,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Falafel Restaurant
4,M1J,Scarborough,Scarborough Village,43.7464,-79.2323,1,Spa,Grocery Store,Event Space,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Falafel Restaurant,Dog Run


In [68]:
toronto_merged.shape

(101, 16)

And finally, to visualize the results:

In [84]:
address = 'Toronto, Ontario, Canada'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(f'The geographical coordinates of Toronto are: {latitude}, {longitude}')

The geographical coordinates of Toronto are: 43.653963, -79.387207


In [85]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, pcode, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['PostalCode'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(f'{str(pcode)}: {str(poi)} -> Cluster {str(cluster)}', parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)

map_clusters

### Conclusion:
Looking at the cluster assignments, it is evident a large number of areas fall under one cluster, specifically Cluster index 1, followed by assignments for Cluster index 0. Perhaps, we could increase the number of Clusters to get proper segmentation of the data and to do further analysis. But we will work on it and try different approaches other than K-means clustering in the coming week.

Nevertheless, Folium combined with Foursquare API has proven to be a powerful tool to work with geographical data..